In [ ]:
import copy
import numpy as np
import pandas as pd
import scipy.stats
import scipy.integrate
import json
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from IPython.display import clear_output, HTML

In [ ]:
def muskingum_matrix(startnodes, endnodes, alpha, beta, chi, gamma, indegree):
    m = startnodes.size
    n = endnodes.size
    indegree_t = indegree.copy()

    A = np.zeros((n, n), dtype=np.float64)
    B = np.zeros((n, n), dtype=np.float64)
    
    # Simulate output
    for k in range(m):
        startnode = startnodes[k]
        endnode = endnodes[startnode]
        while(indegree_t[startnode] == 0):
            alpha_i = alpha[startnode]
            beta_i = beta[startnode]
            chi_i = chi[startnode]
            gamma_i = gamma[startnode]
            alpha_j = alpha[endnode]
            beta_j = beta[endnode]
            # System matrix
            A[startnode, startnode] = chi_i
            B[startnode, startnode] = gamma_i
            # Add outflow to inflow at endnode
            if startnode != endnode:
                A[endnode, startnode] += beta_j
                A[endnode] += alpha_j * A[startnode]
                B[endnode] += alpha_j * B[startnode]
            indegree_t[endnode] -= 1
            startnode = endnode
            endnode = endnodes[startnode]
    return A, B

In [ ]:
with open('../NYC_2022/new_year_creek_full.json') as nyc:
    d = json.load(nyc)

In [ ]:
node_ids = [i['uid'] for i in d['nodes']]

In [ ]:
link_ids = [i['uid'] for i in d['links']]
reach_ids = [link_id[2:] for link_id in link_ids]

In [ ]:
source_node_ids = [i['source_uid'] for i in d['links']]
target_node_ids = [i['target_uid'] for i in d['links']]

In [ ]:
startnodes = np.asarray([node_ids.index(i) if i in node_ids
                         else -1 for i in source_node_ids])
endnodes = np.asarray([node_ids.index(i) if i in node_ids 
                       else -1 for i in target_node_ids])

In [ ]:
self_loops = []

for i in range(len(startnodes)):
    if endnodes[i] == -1:
        self_loops.append(i)
        endnodes[i] = startnodes[i]
        
indegree = np.bincount(endnodes.ravel(), minlength=startnodes.size)

for self_loop in self_loops:
    indegree[self_loop] -= 1

In [ ]:
link_lengths = np.asarray([i['length'] for i in d['links']])

In [ ]:
n = startnodes.size

#X = 0.29 * np.ones(n)
#K = 1.3 * link_lengths
X = 0.29 * np.ones(n)
K = 3600. * np.ones(n)
dt = 3600.
alpha = (dt - 2 * K * X) / (2 * K * (1 - X) + dt)
beta = (dt + 2 * K * X) / (2 * K * (1 - X) + dt)
chi = (2 * K * (1 - X) - dt) / (2 * K * (1 - X) + dt)
gamma = dt / (K * (1 - X) + dt / 2)

In [ ]:
startnodes = startnodes[(indegree == 0)]
A, B = muskingum_matrix(startnodes, endnodes, alpha, beta, chi, gamma, indegree)

In [ ]:
# Load forcing data

In [ ]:
date_range = pd.date_range('20230501', '20230523', freq='6H').strftime('%Y%m%d%H')[:-1]
models = [f'medium_range_{i}' for i in range(1,8)]
site = '5559104'
site_index = [link[2:] for link in link_ids].index(site)
priors = np.ones(len(models)) / len(models)
priors = dict(zip(models, priors))

In [ ]:
upstream1 = pd.read_csv('../nyc2022/scalar_USGS_08111085_flow_Learned_P1H_HIS.csv')

obs = upstream1.set_index(pd.to_datetime(upstream1['Unnamed: 0']).values)
obs = obs.loc['20230430':'20230601']
upstream1_cms = obs['299705_00060'].tz_localize('US/Central') / 35.314666212661
upstream1_cms = upstream1_cms.tz_convert('UTC')

In [ ]:
upstream2 = pd.read_csv('../nyc2022/scalar_USGS_08111090_flow_Learned_P1H_HIS.csv')

obs = upstream2.set_index(pd.to_datetime(upstream1['Unnamed: 0']).values)
obs = obs.loc['20230430':'20230601']
upstream2_cms = obs['300306_00060'].tz_localize('US/Central') / 35.314666212661
upstream2_cms = upstream2_cms.tz_convert('UTC')

In [ ]:
Q = 2 * np.eye(n)
P = 1 * np.eye(n)
R = 0.01 * np.eye(1)

In [ ]:
upstream1 = reach_ids.index('5559138')
upstream2 = reach_ids.index('5559076')
obs_point = reach_ids.index(site)

In [ ]:
H = np.zeros((1, n))
H[:,obs_point] = 1      # Observation point

In [ ]:
obs = pd.read_csv('08111110_discharge.txt', sep='\t', skiprows=29, header=None)
obs = obs.set_index(pd.to_datetime(obs[2]).values)
obs = obs.loc['20230430':'20230601']
obs_cms = 0.0283168 * obs[4].tz_localize('US/Central')
obs_cms = obs_cms.tz_convert('UTC')
obs_cms = obs_cms.resample('1h').interpolate()

# No sensors

In [ ]:
output = {}
uncertainty = {}

for date_index, date in enumerate(date_range):
    output[date] = {}
    uncertainty[date] = {}
    start_datetime = pd.to_datetime(date, format='%Y%m%d%H').tz_localize('UTC')

    end_datetime = start_datetime + pd.Timedelta(hours=204)

    for model in models:
        qBucket = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_{model}_qBucket.csv')
        qSfcLatRunoff = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_{model}_qSfcLatRunoff.csv')
        streamflow = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_{model}_streamflow.csv')

        qBucket_da = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_data_assimilation_qBucket.csv')
        qSfcLatRunoff_da = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_data_assimilation_qSfcLatRunoff.csv')
        streamflow_da = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_data_assimilation_streamflow.csv')

        qBucket['time'] = pd.to_datetime(qBucket['time'])
        qBucket = qBucket.set_index('time').tz_localize('UTC')
        qBucket_da['time'] = pd.to_datetime(qBucket_da['time'])
        qBucket_da = qBucket_da.set_index('time').tz_localize('UTC')

        qSfcLatRunoff['time'] = pd.to_datetime(qSfcLatRunoff['time'])
        qSfcLatRunoff = qSfcLatRunoff.set_index('time').tz_localize('UTC')
        qSfcLatRunoff_da['time'] = pd.to_datetime(qSfcLatRunoff_da['time'])
        qSfcLatRunoff_da = qSfcLatRunoff_da.set_index('time').tz_localize('UTC')

        streamflow['time'] = pd.to_datetime(streamflow['time'])
        streamflow = streamflow.set_index('time').tz_localize('UTC')
        streamflow_da['time'] = pd.to_datetime(streamflow_da['time'])
        streamflow_da = streamflow_da.set_index('time').tz_localize('UTC')
        
        T = len(qBucket)
        assert T == len(qSfcLatRunoff)
        #assert T == len(streamflow)
        
        if date_index == 0:
            o_t_prev = streamflow_da[reach_ids].iloc[0].values
            p_t_prev = qSfcLatRunoff_da[reach_ids].iloc[0].values + qBucket_da[reach_ids].iloc[0].values
            #P_t_prev = P.copy()
            P_t_prev = np.diag(o_t_prev) * 0.1
        else:
            try:
                o_t_prev = output[date_range[date_index - 1]][model].loc[start_datetime].values
                p_t_prev = qSfcLatRunoff_da[reach_ids].loc[start_datetime].values + qBucket_da[reach_ids].loc[start_datetime].values
                P_t_prev = uncertainty[date_range[date_index - 1]][model][start_datetime]
            except:
                o_t_prev = output[date_range[date_index - 2]][model].loc[start_datetime].values
                p_t_prev = qSfcLatRunoff_da[reach_ids].loc[start_datetime].values + qBucket_da[reach_ids].loc[start_datetime].values
                P_t_prev = uncertainty[date_range[date_index - 2]][model][start_datetime]

        O_t = {}
        P_t = {}

        # ### DA ###
        # dz = z[start_datetime] - H @ o_t_prev
        # K = P_t_prev @ H.T @ np.linalg.inv(H @ P_t_prev @ H.T + R)
        # o_t_prev = o_t_prev + K @ dz
        # P_t_prev = (np.eye(n) - K @ H) @ P_t_prev

        O_t[start_datetime] = o_t_prev
        P_t[start_datetime] = P_t_prev
        
        for t in range(T):
            previous_datetime = start_datetime + datetime.timedelta(hours=t)
            next_datetime = start_datetime + datetime.timedelta(hours=t+1)
            o_t_next = A @ o_t_prev + B @ p_t_prev
            Q = np.diag(o_t_next) * 0.1
            P_t_next = A @ P_t_prev @ A.T + Q

            p_t_prev = qSfcLatRunoff[reach_ids].loc[next_datetime].values + qBucket[reach_ids].loc[next_datetime].values

            O_t[next_datetime] = o_t_next
            o_t_prev = o_t_next
            P_t[next_datetime] = P_t_next
            P_t_prev = P_t_next
    
        O_t = pd.DataFrame.from_dict(O_t, orient='index')
        O_t.columns = reach_ids
                
        output[date][model] = O_t
        uncertainty[date][model] = P_t

# With sensors

In [ ]:
H = np.zeros((3, n))
H[0,upstream1] = 1 
H[1,upstream2] = 1 
H[2,obs_point] = 1      # Observation point

In [ ]:
Q = 2 * np.eye(n)
P = 1 * np.eye(n)
R = 0.01 * np.eye(3)

In [ ]:
z = pd.concat([upstream1_cms, upstream2_cms, obs_cms], axis=1)

In [ ]:
plt.plot(z)

In [ ]:
output_up = {}
uncertainty_up = {}
K_gain = {}

for date_index, date in enumerate(date_range):
    output_up[date] = {}
    uncertainty_up[date] = {}
    K_gain[date] = {}
    start_datetime = pd.to_datetime(date, format='%Y%m%d%H').tz_localize('UTC')

    for model in models:
        qBucket = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_{model}_qBucket.csv')
        qSfcLatRunoff = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_{model}_qSfcLatRunoff.csv')
        streamflow = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_{model}_streamflow.csv')

        qBucket_da = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_data_assimilation_qBucket.csv')
        qSfcLatRunoff_da = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_data_assimilation_qSfcLatRunoff.csv')
        streamflow_da = pd.read_csv(f'../nyc2022/export_20230501-27/{date}/{date}_data_assimilation_streamflow.csv')

        qBucket['time'] = pd.to_datetime(qBucket['time'])
        qBucket = qBucket.set_index('time').tz_localize('UTC')
        qBucket_da['time'] = pd.to_datetime(qBucket_da['time'])
        qBucket_da = qBucket_da.set_index('time').tz_localize('UTC')

        qSfcLatRunoff['time'] = pd.to_datetime(qSfcLatRunoff['time'])
        qSfcLatRunoff = qSfcLatRunoff.set_index('time').tz_localize('UTC')
        qSfcLatRunoff_da['time'] = pd.to_datetime(qSfcLatRunoff_da['time'])
        qSfcLatRunoff_da = qSfcLatRunoff_da.set_index('time').tz_localize('UTC')

        streamflow['time'] = pd.to_datetime(streamflow['time'])
        streamflow = streamflow.set_index('time').tz_localize('UTC')
        streamflow_da['time'] = pd.to_datetime(streamflow_da['time'])
        streamflow_da = streamflow_da.set_index('time').tz_localize('UTC')
        
        T = len(qBucket)
        assert T == len(qSfcLatRunoff)
        #assert T == len(streamflow)
        
        if date_index == 0:
            o_t_prev = streamflow_da[reach_ids].iloc[0].values
            p_t_prev = qSfcLatRunoff_da[reach_ids].iloc[0].values + qBucket_da[reach_ids].iloc[0].values
            #P_t_prev = P.copy()
            P_t_prev = np.diag(o_t_prev) * 0.1
        else:
            try:
                o_t_prev = output_up[date_range[date_index - 1]][model].loc[start_datetime].values
                p_t_prev = qSfcLatRunoff_da[reach_ids].iloc[0].values + qBucket_da[reach_ids].iloc[0].values
                P_t_prev = uncertainty_up[date_range[date_index - 1]][model][start_datetime]
            except:
                o_t_prev = output_up[date_range[date_index - 2]][model].loc[start_datetime].values
                p_t_prev = qSfcLatRunoff_da[reach_ids].iloc[0].values + qBucket_da[reach_ids].iloc[0].values
                P_t_prev = uncertainty_up[date_range[date_index - 2]][model][start_datetime]

        O_t = {}
        P_t = {}

        ### DA ###
        dz = z.loc[start_datetime].values - H @ o_t_prev
        K = P_t_prev @ H.T @ np.linalg.inv(H @ P_t_prev @ H.T + R)
        o_t_prev = o_t_prev + K @ dz
        gain = K @ dz
        P_t_prev = (np.eye(n) - K @ H) @ P_t_prev

        O_t[start_datetime] = o_t_prev
        P_t[start_datetime] = P_t_prev
       
        for t in range(T):
            previous_datetime = start_datetime + datetime.timedelta(hours=t)
            next_datetime = start_datetime + datetime.timedelta(hours=t+1)
            o_t_next = A @ o_t_prev + B @ p_t_prev
            Q = np.diag(o_t_next) * 0.1
            P_t_next = A @ P_t_prev @ A.T + Q

            p_t_prev = qSfcLatRunoff[reach_ids].loc[next_datetime].values + qBucket[reach_ids].loc[next_datetime].values

            O_t[next_datetime] = o_t_next
            o_t_prev = o_t_next
            P_t[next_datetime] = P_t_next
            P_t_prev = P_t_next
    
        O_t = pd.DataFrame.from_dict(O_t, orient='index')
        O_t.columns = reach_ids
                
        output_up[date][model] = O_t
        uncertainty_up[date][model] = P_t
        K_gain[date][model] = gain

In [ ]:
mean_sensor = {}

for date, data in output_up.items():
    for model, df in data.items():
        mean_sensor[date] = df.iloc[0].to_dict()

In [ ]:
df = pd.DataFrame(mean_sensor)
df_transposed = df.transpose()
df_transposed.to_csv('stremflow_with_sensor.csv')

In [ ]:
mean_without_sensor = {}

for date, data in output.items():
    for model, df in data.items():
        mean_without_sensor[date] = df.iloc[0].to_dict()

In [ ]:
df = pd.DataFrame(mean_sensor)
df_transposed = df.transpose()
df_transposed.to_csv('stremflow_without_sensor.csv')

In [ ]:
std_sensor = {}

for date in date_range:
    std_sensor[date] = {}
    for timestamp, cov in uncertainty_up[date][model].items():
        std_sensor[date] = np.sqrt(np.diag(cov))
        break

In [ ]:
std_df = pd.DataFrame(std_sensor)
std_transposed = std_df.transpose()
std_transposed.columns = O_t.columns
std_transposed.to_csv('std_with_sensor.csv')

In [ ]:
std = {}

for date in date_range:
    std[date] = {}
    for timestamp, cov in uncertainty[date][model].items():
        std[date] = np.sqrt(np.diag(cov))
        break

In [ ]:
std_df = pd.DataFrame(std)
std_transposed = std_df.transpose()
std_transposed.columns = O_t.columns
std_transposed.to_csv('std_without_sensor.csv')

In [ ]:
difference = {}

for date in date_range:
    difference[date] = std[date] - std_sensor[date]

In [ ]:
import geopandas as gpd
import matplotlib.colors as mcolors
import datetime
streams = gpd.read_file('../NYC_2022/data/streams.shp')
streams = streams.set_index(streams['ID'].astype(int))
gage_shp = gpd.read_file('../NYC_2022/data/Gauge.shp')

In [ ]:
streams['ID'] = streams['ID'].astype(int)
streams = streams.set_index('ID')

In [ ]:
for date in date_range:
    streams_temp = streams.copy()

    value = std[date]

    index = O_t.columns.astype(int)
    K_gain_series = pd.Series(value, index=pd.Index(index, dtype=int), name='K_gain_medium_range_5')
    K_gain_df = K_gain_series.reset_index()
    K_gain_df.columns = ['ID', 'K_gain_medium_range_5']

    streams_temp = streams_temp.reset_index()
    streams_temp = streams_temp.merge(K_gain_df, on='ID', how='left')

    date_obj = datetime.datetime.strptime(date, '%Y%m%d%H')
    readable_date = date_obj.strftime('%Y-%m-%d')


    fig, ax = plt.subplots()
    fig.patch.set_visible(False)

    vmin = 0
    vmax = 5
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vmax=vmax, vcenter=2.5)

    # Plotting
    streams_temp.plot(ax=ax, column='K_gain_medium_range_5', legend=True, cmap='YlOrRd', norm=norm)
    gage_shp.to_crs(streams_temp.crs).plot(ax=ax, color='darkgrey', zorder=2, markersize=100)
    ax.text(0.1, 0.9, f'{readable_date}', transform=ax.transAxes)
    ax.axis('off')
    
    # Save the plot
    plt.savefig(f'./std_nosensor/img_{date}.jpg', bbox_inches='tight', dpi=200)

In [ ]:
for date in date_range:
    streams_temp = streams.copy()

    value = std_sensor[date]

    index = O_t.columns.astype(int)
    K_gain_series = pd.Series(value, index=pd.Index(index, dtype=int), name='K_gain_medium_range_5')
    K_gain_df = K_gain_series.reset_index()
    K_gain_df.columns = ['ID', 'K_gain_medium_range_5']

    streams_temp = streams_temp.reset_index()
    streams_temp = streams_temp.merge(K_gain_df, on='ID', how='left')

    date_obj = datetime.datetime.strptime(date, '%Y%m%d%H')
    readable_date = date_obj.strftime('%Y-%m-%d')


    fig, ax = plt.subplots()
    fig.patch.set_visible(False)

    vmin = 0
    vmax = 5
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vmax=vmax, vcenter=2.5)

    # Plotting
    streams_temp.plot(ax=ax, column='K_gain_medium_range_5', legend=True, cmap='YlOrRd', norm=norm)
    gage_shp.to_crs(streams_temp.crs).plot(ax=ax, color='green', zorder=2, markersize=100)
    ax.text(0.1, 0.9, f'{readable_date}', transform=ax.transAxes)
    ax.axis('off')
    
    # Save the plot
    plt.savefig(f'./std_withsensor/img_{date}.jpg', bbox_inches='tight', dpi=200)

In [ ]:
for date in date_range:
    streams_temp = streams.copy()

    value = difference[date]

    index = O_t.columns.astype(int)
    K_gain_series = pd.Series(value, index=pd.Index(index, dtype=int), name='K_gain_medium_range_5')
    K_gain_df = K_gain_series.reset_index()
    K_gain_df.columns = ['ID', 'K_gain_medium_range_5']

    streams_temp = streams_temp.reset_index()
    streams_temp = streams_temp.merge(K_gain_df, on='ID', how='left')

    date_obj = datetime.datetime.strptime(date, '%Y%m%d%H')
    readable_date = date_obj.strftime('%Y-%m-%d')


    fig, ax = plt.subplots()
    fig.patch.set_visible(False)

    vmin = 0
    vmax = 1
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vmax=vmax, vcenter=0.5)

    # Plotting
    streams_temp.plot(ax=ax, column='K_gain_medium_range_5', legend=True, cmap='YlOrRd', norm=norm)
    gage_shp.to_crs(streams_temp.crs).plot(ax=ax, color='darkgrey', zorder=2, markersize=100)
    ax.text(0.1, 0.9, f'{readable_date}', transform=ax.transAxes)
    ax.axis('off')
    
    # Save the plot
    plt.savefig(f'./std_diff/img_{date}.jpg', bbox_inches='tight', dpi=200)